In [1]:
!nvidia-smi

Sat May 22 13:11:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%bash
wget -q https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb

dpkg -i nvidia-machine-learning-repo-*.deb
apt-get -qq update

sudo apt-get -qq install libnvinfer5 #libnvinfer6=6.0.1-1+cuda10.1

pip install -q tensorflow-gpu==2.0.0

Selecting previously unselected package nvidia-machine-learning-repo-ubuntu1804.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb ...
Unpacking nvidia-machine-learning-repo-ubuntu1804 (1.0.0-1) ...
Setting up nvidia-machine-learning-repo-ubuntu1804 (1.0.0-1) ...
Selecting previously unselected package libnvinfer5.
(Reading database ... 160709 files and directories currently installed.)
Preparing to unpack .../libnvinfer5_5.1.5-1+cuda10.1_amd64.deb ...
Unpacking libnvinfer5 (5.1.5-1+cuda10.1) ...
Setting up libnvinfer5 (5.1.5-1+cuda10.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



W: Target Packages (Packages) is configured multiple times in /etc/apt/sources.list.d/nvidia-machine-learning.list:1 and /etc/apt/sources.list.d/nvidia-ml.list:1
W: Target Packages (Packages) is configured multiple times in /etc/apt/sources.list.d/nvidia-machine-learning.list:1 and /etc/apt/sources.list.d/nvidia-ml.list:1
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
W: Target Packages (Packages) is configured multiple times in /etc/apt/sources.list.d/nvidia-machine-learning.list:1 and /etc/apt/sources.list.d/nvidia-ml.list:1
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll h

In [4]:
# check TensorRT version
import tensorflow as tf
print("TensorFlow version: ", tf.version.VERSION)
print("TensorRT version: ")
!dpkg -l | grep nvinfer

TensorFlow version:  2.0.0
TensorRT version: 
ii  libnvinfer5                             5.1.5-1+cuda10.1                    amd64        TensorRT runtime libraries


In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop

In [11]:
vgg19_model= VGG19(weights='imagenet',include_top=False,input_shape=(224,224,3))

for layer in vgg19_model.layers:
  layer.trainable = False
vgg19_model.summary()  

80142336/80134624 [==============================] - 1s 0us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
______________________________________________________________

In [12]:
x = layers.Flatten()(vgg19_model.output)
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.3
x = layers.Dropout(0.3)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model(vgg19_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [14]:
# Add our data-augmentation parameters to ImageDataGenerator
train_dir='/content/drive/MyDrive/datacov/train'
validation_dir='/content/drive/MyDrive/datacov/val'
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (224,224))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (224,224))

Found 2232 images belonging to 2 classes.
Found 249 images belonging to 2 classes.


In [15]:
history = model.fit_generator(
            train_generator,
            validation_data = validation_generator,
            epochs = 35
            )

Epoch 1/35
112/112 [==============================] - 70s 628ms/step - loss: 0.8634 - acc: 0.6165 - val_loss: 0.4625 - val_acc: 0.7751
Epoch 2/35
112/112 [==============================] - 62s 551ms/step - loss: 0.5455 - acc: 0.7384 - val_loss: 0.3689 - val_acc: 0.8434
Epoch 3/35
112/112 [==============================] - 62s 550ms/step - loss: 0.5187 - acc: 0.7639 - val_loss: 0.3958 - val_acc: 0.8072
Epoch 4/35
112/112 [==============================] - 62s 550ms/step - loss: 0.4335 - acc: 0.7944 - val_loss: 0.4181 - val_acc: 0.7831
Epoch 5/35
112/112 [==============================] - 62s 550ms/step - loss: 0.4394 - acc: 0.8047 - val_loss: 0.2953 - val_acc: 0.8675
Epoch 6/35
112/112 [==============================] - 61s 549ms/step - loss: 0.4181 - acc: 0.8082 - val_loss: 0.3080 - val_acc: 0.8755
Epoch 7/35
112/112 [==============================] - 62s 550ms/step - loss: 0.3836 - acc: 0.8320 - val_loss: 0.3055 - val_acc: 0.8514
Epoch 8/35
112/112 [==============================] - 6

In [36]:
img0 = image.load_img('/content/drive/MyDrive/datacov/val/covid/Covid (1018).png',target_size=(224, 224))
x0 = image.img_to_array(img0)
x0 = np.expand_dims(x0, axis=0)
preds0 = model.predict(x0)
print(preds0[0])
img1 = image.load_img('/content/drive/MyDrive/datacov/val/no_covid/Non-Covid (101).png',target_size=(224, 224))
x1 = image.img_to_array(img1)
x1 = np.expand_dims(x1, axis=0)
preds1 = model.predict(x1)
print(preds1[0])

[0.]
[1.]


In [16]:
tf.saved_model.save(model,'covid_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: covid_model/assets


In [17]:
!saved_model_cli show --all --dir covid_model/


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_2'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 224, 224, 3)
        name: serving_default_input_2:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_3'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict


In [4]:
import os 
import glob
import random
img_paths=[]
for dir_path in glob.glob('/content/drive/MyDrive/datacov/val/*'):
  for file_path in glob.glob(os.path.join(dir_path,"*.png")):
    img_paths.append(file_path)
print(img_paths)
length=len(img_paths)
shuffled_img_paths=random.sample(img_paths, length)
print(shuffled_img_paths)

['/content/drive/MyDrive/datacov/val/no_covid/Non-Covid (8).png', '/content/drive/MyDrive/datacov/val/no_covid/Non-Covid (824).png', '/content/drive/MyDrive/datacov/val/no_covid/Non-Covid (629).png', '/content/drive/MyDrive/datacov/val/no_covid/Non-Covid (810).png', '/content/drive/MyDrive/datacov/val/no_covid/Non-Covid (907).png', '/content/drive/MyDrive/datacov/val/no_covid/Non-Covid (1124).png', '/content/drive/MyDrive/datacov/val/no_covid/Non-Covid (799).png', '/content/drive/MyDrive/datacov/val/no_covid/Non-Covid (702).png', '/content/drive/MyDrive/datacov/val/no_covid/Non-Covid (705).png', '/content/drive/MyDrive/datacov/val/no_covid/Non-Covid (285).png', '/content/drive/MyDrive/datacov/val/no_covid/Non-Covid (904).png', '/content/drive/MyDrive/datacov/val/no_covid/Non-Covid (323).png', '/content/drive/MyDrive/datacov/val/no_covid/Non-Covid (140).png', '/content/drive/MyDrive/datacov/val/no_covid/Non-Covid (932).png', '/content/drive/MyDrive/datacov/val/no_covid/Non-Covid (380).p

In [5]:
import numpy as np
from tensorflow.keras.preprocessing import image
def batch_input(batch_size=8):
  batched_input=np.zeros((batch_size,224,224,3),dtype=np.float32)
  for i in range(batch_size):
    img_path=shuffled_img_paths[i]
    img=image.load_img(img_path,target_size=(224,224))
    x=image.img_to_array(img)
    x=np.expand_dims(x,axis=0)
    batched_input[i,:]=x

    batched_input=tf.constant(batched_input) #convert to tf eagertensor
    return batched_input


In [6]:
batched_input=batch_input(batch_size=32)
batched_input.shape

TensorShape([32, 224, 224, 3])

In [12]:
from tensorflow.python.saved_model import tag_constants
def load_tf_saved_model(input_saved_model_dir):
  print(f'Loading saved model{input_saved_model_dir}....')
  saved_model_loaded=tf.saved_model.load(input_saved_model_dir,tags=[tag_constants.SERVING])
  return saved_model_loaded

In [32]:
saved_model=load_tf_saved_model('covid_model')

Loading saved modelcovid_model....


In [33]:
infer=saved_model.signatures['serving_default']
print(infer.structured_outputs)

{'dense_3': TensorSpec(shape=(None, 1), dtype=tf.float32, name='dense_3')}


In [7]:
import time
def predict_and_benchmark_throughput(batched_input, infer, N_warmup_run=50, N_run=1000):

  elapsed_time = []
  all_preds = []
  batch_size = batched_input.shape[0]

  for i in range(N_warmup_run):
    labeling = infer(batched_input)
    preds = labeling['dense_3'].numpy()

  for i in range(N_run):
    start_time = time.time()

    labeling = infer(batched_input)

    preds = labeling['dense_3'].numpy()

    end_time = time.time()

    elapsed_time = np.append(elapsed_time, end_time - start_time)
    
    all_preds.append(preds)

    if i % 50 == 0:
      print('Steps {}-{} average: {:4.1f}ms'.format(i, i+50, (elapsed_time[-50:].mean()) * 1000))

  print('Throughput: {:.0f} images/s'.format(N_run * batch_size / elapsed_time.sum()))
  return all_preds

In [44]:
# without tensorrt optimized model predcition speed
all_preds=predict_and_benchmark_throughput(batched_input,infer,N_warmup_run=50,N_run=1000)

Steps 0-50 average: 159.7ms
Steps 50-100 average: 160.6ms
Steps 100-150 average: 162.4ms
Steps 150-200 average: 162.5ms
Steps 200-250 average: 161.0ms
Steps 250-300 average: 159.4ms
Steps 300-350 average: 158.5ms
Steps 350-400 average: 158.1ms
Steps 400-450 average: 157.8ms
Steps 450-500 average: 158.0ms
Steps 500-550 average: 158.5ms
Steps 550-600 average: 159.4ms
Steps 600-650 average: 159.8ms
Steps 650-700 average: 160.0ms
Steps 700-750 average: 159.9ms
Steps 750-800 average: 159.7ms
Steps 800-850 average: 159.4ms
Steps 850-900 average: 159.1ms
Steps 900-950 average: 158.8ms
Steps 950-1000 average: 158.9ms
Throughput: 201 images/s


In [52]:
#Testing without tensorrt optimized model with the same image
test_im = image.load_img('/content/drive/MyDrive/datacov/val/covid/Covid (1018).png',target_size=(224, 224))
x_test1= image.img_to_array(test_im)
x_test1= np.expand_dims(x_test1, axis=0)
x_test1=tf.constant(x_test1)
labeling = infer(x_test1)
preds = labeling['dense_3'].numpy()
print(preds)


[[0.]]


In [9]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt
def convert_to_trt_graph_and_save(precision_mode='float16',input_saved_model_dir='covid_model',calibration_data=batched_input):
  if precision_mode=='float16':
    precision_mode=trt.TrtPrecisionMode.FP16
    converted_save_suffix='_TFTRT_FP16'
  output_saved_model_dir=input_saved_model_dir+converted_save_suffix
  conversion_params = trt.DEFAULT_TRT_CONVERSION_PARAMS._replace(
  precision_mode=precision_mode,
  max_workspace_size_bytes=8000000000)
  converter=trt.TrtGraphConverterV2(input_saved_model_dir=input_saved_model_dir,conversion_params=conversion_params)

  print(f'Converting {input_saved_model_dir} to TF-TRT graph precision mode {precision_mode} ....')
  converter.convert()
  print(f'Saving converted model to {output_saved_model_dir}')
  converter.save(output_saved_model_dir=output_saved_model_dir)
  print('complete')
  

In [11]:
convert_to_trt_graph_and_save(precision_mode='float16',input_saved_model_dir='covid_model')

INFO:tensorflow:Linked TensorRT version: (5, 1, 5)
INFO:tensorflow:Loaded TensorRT version: (5, 1, 5)
INFO:tensorflow:Running against TensorRT version 5.1.5
Converting covid_model to TF-TRT graph precision mode FP16 ....
Saving converted model to covid_model_TFTRT_FP16
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: covid_model_TFTRT_FP16/assets
complete


In [13]:
saved_model_trt=load_tf_saved_model('/content/covid_model_TFTRT_FP16')

Loading saved model/content/covid_model_TFTRT_FP16....


In [14]:
infer_trt=saved_model_trt.signatures['serving_default']

In [15]:
#tensorrt optimized model predcition speed
all_preds_trt=predict_and_benchmark_throughput(batched_input,infer_trt,N_warmup_run=50,N_run=1000)

Steps 0-50 average: 59.9ms
Steps 50-100 average: 59.3ms
Steps 100-150 average: 59.5ms
Steps 150-200 average: 59.9ms
Steps 200-250 average: 60.9ms
Steps 250-300 average: 60.9ms
Steps 300-350 average: 60.9ms
Steps 350-400 average: 60.9ms
Steps 400-450 average: 60.8ms
Steps 450-500 average: 60.8ms
Steps 500-550 average: 60.9ms
Steps 550-600 average: 60.4ms
Steps 600-650 average: 60.1ms
Steps 650-700 average: 59.5ms
Steps 700-750 average: 59.3ms
Steps 750-800 average: 58.9ms
Steps 800-850 average: 59.0ms
Steps 850-900 average: 58.7ms
Steps 900-950 average: 58.5ms
Steps 950-1000 average: 58.5ms
Throughput: 535 images/s


In [18]:
#Testing on tensorrt fp16 optimized model with the same image
test_im_trt = image.load_img('/content/drive/MyDrive/datacov/val/covid/Covid (1018).png',target_size=(224, 224))
x_test1_trt= image.img_to_array(test_im_trt)
x_test1_trt= np.expand_dims(x_test1_trt, axis=0)
x_test1_trt=tf.constant(x_test1_trt)
labeling_trt = infer_trt(x_test1_trt)
preds_trt = labeling_trt['dense_3'].numpy()
print(preds_trt)


[[0.]]
